In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from tqdm.keras import TqdmCallback

2024-05-22 03:05:02.383401: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-22 03:05:02.410167: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 03:05:02.865763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## 红球模型

In [2]:
# 红蓝球超参
epochs=1000
batch_size=10000
layer=10

# 优化器
adam_optimizer = Adam(learning_rate=1.0)

In [3]:
# 读取数据
data = pd.read_excel('双色球.xlsx')
input_features = data[['Ball1', 'Ball2', 'Ball3', 'Ball4', 'Ball5', 'Ball6']]

# One-Hot编码
label_binarizer = LabelBinarizer()
label_binarizer.fit(range(1, 34))
input_features_one_hot = label_binarizer.transform(input_features.values.flatten()).reshape(-1, 6, 33).sum(axis=1)

In [4]:
# 构建红球模型
model = Sequential([
    Input(shape=(33,)),
    Dense(128, activation='relu'),
    *[Dense(64, activation='relu') for _ in range(layer)],
    Dense(33, activation='sigmoid')
])

# 编译红球模型
model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
# 红球模型训练
model.fit(input_features_one_hot, input_features_one_hot, epochs=epochs, batch_size=batch_size,           
            verbose=0, callbacks=[TqdmCallback(verbose=1), 
                                    EarlyStopping(monitor='loss', patience=layer)
                                    ])

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

In [6]:
# 红球预测
predictions = model.predict(input_features_one_hot[-1].reshape(1, -1))
predicted_indices = np.argsort(predictions[0])[::-1][:6]  # 选择最高的6个概率
predicted_balls = np.array(range(1, 34))[predicted_indices]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


## 蓝球模型

In [7]:
# 读取数据
data = pd.read_excel('双色球.xlsx')
blue_balls = data['BlueBall'].values.reshape(-1, 1)

# 归一化
scaler = MinMaxScaler(feature_range=(0, 1))
blue_balls_scaled = scaler.fit_transform(blue_balls)

In [11]:
# 构建蓝球模型
model = Sequential()
model.add(Input(shape=(1,)))
model.add(Dense(128, activation='relu'))
for _ in range(layer):
    model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # 输出层，1个输出

# 编译建蓝球模型
model.compile(loss='mean_squared_error', optimizer='adam')

In [12]:
# 蓝球模型训练
model.fit(blue_balls_scaled, 
            blue_balls_scaled, 
            epochs=epochs, 
            batch_size=batch_size,
            verbose=0,
            callbacks=[TqdmCallback(verbose=1)]
            )

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

In [ ]:
# 蓝球预测
prediction = model.predict(blue_balls_scaled[-1].reshape(1, -1))
predicted_ball = scaler.inverse_transform(prediction).astype(int)

## 预测结果

In [ ]:
# 输出预测结果
print('Predicted Red Balls:', predicted_balls)
print('Predicted Blue Ball:', predicted_ball.flatten()[0])